In [1]:
import pandas as pd
import torch
import numpy as np
import os

from tqdm import tqdm
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics.classification import BinaryF1Score

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn import preprocessing

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda:0'

In [4]:
torch.set_default_device(device)

In [5]:
if device == 'cuda':
    torch.cuda.empty_cache()

In [6]:
df = pd.read_csv('emails.csv')

In [7]:
df

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,Email 5168,2,2,2,3,0,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,Email 5169,35,27,11,2,6,5,151,4,3,...,0,0,0,0,0,0,0,1,0,0
5169,Email 5170,0,0,1,1,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,Email 5171,2,7,1,0,2,1,28,2,0,...,0,0,0,0,0,0,0,1,0,1


In [8]:
#Email No. column replicates index
df = df.drop(columns=['Email No.'])
#Remove duplicate rows
df = df.drop_duplicates()

In [9]:
df

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,2,2,2,3,0,0,32,0,0,5,...,0,0,0,0,0,0,0,0,0,0
5168,35,27,11,2,6,5,151,4,3,23,...,0,0,0,0,0,0,0,1,0,0
5169,0,0,1,1,0,0,11,0,0,1,...,0,0,0,0,0,0,0,0,0,1
5170,2,7,1,0,2,1,28,2,0,8,...,0,0,0,0,0,0,0,1,0,1


In [10]:
#Imbalanced data - oversampling may be needed?
df['Prediction'].value_counts()

Prediction
0    3170
1    1461
Name: count, dtype: int64

In [11]:
#Outlier or spam email?
df.max(axis=None)

2327

In [12]:
df[['Is_Legit', 'Is_Spam']] = pd.get_dummies(df['Prediction'])
df.drop(columns='Prediction', inplace=True)

In [13]:
df

,the,to,ect,and,for,of,a,you,hou,in,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Is_Legit,Is_Spam
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,True,False
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,1,0,True,False
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,True,False
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,True,False
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,1,0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,2,2,2,3,0,0,32,0,0,5,...,0,0,0,0,0,0,0,0,True,False
5168,35,27,11,2,6,5,151,4,3,23,...,0,0,0,0,0,0,1,0,True,False
5169,0,0,1,1,0,0,11,0,0,1,...,0,0,0,0,0,0,0,0,False,True
5170,2,7,1,0,2,1,28,2,0,8,...,0,0,0,0,0,0,1,0,False,True


In [14]:
validation_df = df.sample(frac=0.1)

In [15]:
#Remove validation data from train df
df = df[~df.isin(validation_df)].dropna().map(lambda x: int(x))

In [16]:
df

,the,to,ect,and,for,of,a,you,hou,in,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Is_Legit,Is_Spam
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,1,0,1,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,1,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,1,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,1,0,1,1,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5167,2,2,2,3,0,0,32,0,0,5,...,0,0,0,0,0,0,0,0,1,0
5169,0,0,1,1,0,0,11,0,0,1,...,0,0,0,0,0,0,0,0,0,1
5170,2,7,1,0,2,1,28,2,0,8,...,0,0,0,0,0,0,1,0,0,1


In [17]:
validation_df = validation_df.sort_index()
validation_df

,the,to,ect,and,for,of,a,you,hou,in,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Is_Legit,Is_Spam
5,4,5,1,4,2,3,45,1,0,16,...,0,0,0,0,0,0,0,0,False,True
15,6,2,1,0,2,0,36,3,1,8,...,0,0,0,0,0,0,0,0,True,False
17,36,21,6,14,7,17,194,25,5,59,...,0,0,0,0,0,0,3,0,False,True
20,0,0,1,1,0,0,15,1,0,2,...,0,0,0,0,0,0,0,0,True,False
22,0,3,6,0,5,0,30,0,2,6,...,0,0,0,0,0,0,0,0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,5,8,10,1,0,0,40,3,1,14,...,0,0,0,0,0,0,0,0,True,False
5136,1,4,1,2,0,1,15,1,0,1,...,0,0,0,0,0,0,1,0,True,False
5146,0,3,2,0,0,0,7,0,0,1,...,0,0,0,0,0,0,0,0,True,False
5158,2,1,1,0,1,1,16,0,1,2,...,0,0,0,0,0,0,1,0,True,False


In [18]:
NUM_CLASSES = 2
HIDDEN_LAYERS = 6000
NUM_EPOCHS = 8
BATCH_SIZE = 128

In [19]:
#Classic DNN
class DNNClassifier(nn.Module):
    def __init__(self, input_layers, hidden_layers, output_layers):
        super().__init__()
        self.sequential_ = nn.Sequential(
            nn.Linear(in_features = input_layers, out_features = hidden_layers),
            #nn.GELU(),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(in_features = hidden_layers, out_features = hidden_layers),
            #nn.GELU(),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(in_features = hidden_layers, out_features = output_layers),
            nn.Sigmoid(),
            #nn.LogSoftmax(dim=1),
        )
    def forward(self, x):
        return self.sequential_(x)

In [20]:
model_ = DNNClassifier(3000, HIDDEN_LAYERS, NUM_CLASSES)
model_ = model_.to(device)

In [21]:
torch.compile(model_)

OptimizedModule(
  (_orig_mod): DNNClassifier(
    (sequential_): Sequential(
      (0): Linear(in_features=3000, out_features=6000, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): Linear(in_features=6000, out_features=6000, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.2, inplace=False)
      (6): Linear(in_features=6000, out_features=2, bias=True)
      (7): Sigmoid()
    )
  )
)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, ~df.columns.isin(['Is_Spam', 'Is_Legit'])], df[['Is_Spam', 'Is_Legit']], test_size = 0.2)

In [23]:
#Minmax scaler is required as a normalization tool to remove outlier effect
scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train.values)
X_test = scaler.fit_transform(X_test.values)

In [24]:
loss_fn = nn.BCELoss()
optimizer = Adam(params = model_.parameters(), lr=1e-4, weight_decay=1e-5)

In [25]:
metric = BinaryF1Score()

In [26]:
X_train_tensor =  torch.from_numpy(X_train).float()
y_train_tensor =  torch.from_numpy(y_train.values).float()
X_test_tensor =  torch.from_numpy(X_test).float()
y_test_tensor =  torch.from_numpy(y_test.values).float()

In [27]:
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

In [28]:
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [29]:
for epoch in tqdm(range(NUM_EPOCHS)):
    train_loss, train_f1 = 0, 0
    for batch, (X, y) in enumerate(train_dl):
        X = X.to(device)
        y = y.to(device)
        y_pred = model_(X)
        loss = loss_fn(y_pred, y)        
        train_loss += loss
        train_f1 += metric(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_dl)
    train_f1 /= len(train_dl)    
    print(f"Train loss: {train_loss:.5f} | Train F1: {train_f1*100:.2f}%")
    
    test_loss, test_f1 = 0, 0
    with torch.inference_mode():         
        for X, y in test_dl:
            X = X.to(device)
            y = y.to(device)
            test_pred = model_(X)     
            test_loss += loss_fn(test_pred, y)
            test_f1 += metric(test_pred, y)
        test_loss /= len(test_dl)
        test_f1 /= len(test_dl)        
        print(f"Test loss: {test_loss:.5f} | Test F1: {test_f1*100:.2f}%\n")


 12%|█▎        | 1/8 [00:00<00:04,  1.46it/s]

Train loss: 0.55834 | Train F1: 69.21%
Test loss: 0.36631 | Test F1: 76.68%



 25%|██▌       | 2/8 [00:01<00:03,  1.97it/s]

Train loss: 0.27987 | Train F1: 84.71%
Test loss: 0.19961 | Test F1: 92.96%



 38%|███▊      | 3/8 [00:01<00:02,  2.21it/s]

...

Train loss: 0.01737 | Train F1: 99.65%
Test loss: 0.65367 | Test F1: 96.40%



 88%|████████▊ | 7/8 [00:03<00:00,  2.50it/s]

Train loss: 0.01322 | Train F1: 99.71%
Test loss: 0.69093 | Test F1: 96.24%



100%|██████████| 8/8 [00:03<00:00,  2.35it/s]

Train loss: 0.01063 | Train F1: 99.72%
Test loss: 0.70683 | Test F1: 96.46%



In [30]:
y_pred_f1 = np.around(test_pred.cpu().numpy()[:, 0], 0)

In [31]:
y_test_f1 = y.cpu().numpy()[:, 0]

In [32]:
f1_score(y_pred_f1, y_test_f1)

0.8717948717948718

In [33]:
#Validation
validation_features = validation_df.loc[:, ~validation_df.columns.isin(['Is_Legit', 'Is_Spam'])]

In [34]:
scaler = preprocessing.MinMaxScaler()
validation_features = scaler.fit_transform(validation_features.values)

In [35]:
validation_pred = model_(torch.Tensor(validation_features).to(device)).cpu().detach().numpy()

In [36]:
validation_df['pred'] = np.around(validation_pred[:, 0], 0)
validation_df.drop(columns='Is_Legit', inplace=True)

In [37]:
validation_df

,the,to,ect,and,for,of,a,you,hou,in,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Is_Spam,pred
5,4,5,1,4,2,3,45,1,0,16,...,0,0,0,0,0,0,0,0,True,1.0
15,6,2,1,0,2,0,36,3,1,8,...,0,0,0,0,0,0,0,0,False,0.0
17,36,21,6,14,7,17,194,25,5,59,...,0,0,0,0,0,0,3,0,True,1.0
20,0,0,1,1,0,0,15,1,0,2,...,0,0,0,0,0,0,0,0,False,0.0
22,0,3,6,0,5,0,30,0,2,6,...,0,0,0,0,0,0,0,0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,5,8,10,1,0,0,40,3,1,14,...,0,0,0,0,0,0,0,0,False,0.0
5136,1,4,1,2,0,1,15,1,0,1,...,0,0,0,0,0,0,1,0,False,0.0
5146,0,3,2,0,0,0,7,0,0,1,...,0,0,0,0,0,0,0,0,False,0.0
5158,2,1,1,0,1,1,16,0,1,2,...,0,0,0,0,0,0,1,0,False,0.0


In [38]:
f1_score(validation_df['pred'], validation_df['Is_Spam'])

0.9611307420494699

In [39]:
#Great F1 score, save model
torch.jit.script(model_).save(os.path.join(os.getcwd(), 'linear_model.pt'))

In [40]:
#1D convolutional NN
#Using the same dataloader and batch settings

In [41]:
NUM_CLASSES = 2
NUM_EPOCHS = 64
BATCH_SIZE = 128

In [42]:
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [43]:
if device == 'cuda':
    torch.cuda.empty_cache()

In [44]:
class CNN1DClassifier(nn.Module):
    def __init__(self, input_channels, length, output_channels, k_size = 3, k_size_pooling = 2):
        super().__init__()
        self.sequential_ = nn.Sequential(
            #Layer1
            nn.Conv1d(in_channels = input_channels, out_channels = length, kernel_size = (k_size), padding=2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.MaxPool1d(kernel_size = (k_size_pooling), stride=k_size_pooling),
            #Layer2
            nn.Conv1d(in_channels = length, out_channels = int(length*2), kernel_size = (k_size), padding=2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.MaxPool1d(kernel_size = (k_size_pooling), stride=k_size_pooling),
            #Dense layer
            nn.Flatten(),
            nn.LazyLinear(out_features = output_channels),#binary
            nn.Sigmoid(),
        )
    def forward(self, x):
        return self.sequential_(x)

In [45]:
model_2 = CNN1DClassifier(input_channels = 1, length = 32, output_channels = 2, k_size = 3, k_size_pooling = 2)

/home/eugen/anaconda3/envs/tfv2/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [46]:
loss_fn = nn.BCELoss()
optimizer = Adam(params = model_2.parameters(), lr=1e-4, weight_decay=1e-5)

In [47]:
metric = BinaryF1Score()

In [48]:
torch.compile(model_2)

OptimizedModule(
  (_orig_mod): CNN1DClassifier(
    (sequential_): Sequential(
      (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(2,))
      (1): GELU(approximate='none')
      (2): Dropout(p=0.2, inplace=False)
      (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(2,))
      (5): GELU(approximate='none')
      (6): Dropout(p=0.2, inplace=False)
      (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Flatten(start_dim=1, end_dim=-1)
      (9): LazyLinear(in_features=0, out_features=2, bias=True)
      (10): Sigmoid()
    )
  )
)

In [49]:
for epoch in tqdm(range(NUM_EPOCHS)):
    train_loss, train_f1 = 0, 0
    for batch, (X, y) in enumerate(train_dl):
        X = X.unsqueeze(dim=1).to(device)
        y = y.unsqueeze(dim=1).to(device)
        y_pred = model_2(X).squeeze(dim=1)
        loss = loss_fn(y_pred, y.squeeze(dim=1))        
        train_loss += loss
        train_f1 += metric(y_pred, y.squeeze(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_dl)
    train_f1 /= len(train_dl)    
    print(f"Train loss: {train_loss:.5f} | Train F1: {train_f1*100:.2f}%")
    
    test_loss, test_f1 = 0, 0
    with torch.inference_mode():         
        for X, y in test_dl:
            X = X.unsqueeze(dim=1).to(device)
            y = y.unsqueeze(dim=1).to(device)
            test_pred = model_2(X).squeeze(dim=1)   
            test_loss += loss_fn(test_pred, y.squeeze(dim=1))
            test_f1 += metric(test_pred, y.squeeze(dim=1))
        test_loss /= len(test_dl)
        test_f1 /= len(test_dl)        
        print(f"Test loss: {test_loss:.5f} | Test F1: {test_f1*100:.2f}%\n")

  2%|▏         | 1/64 [00:00<00:21,  2.97it/s]

Train loss: 0.63451 | Train F1: 67.51%
Test loss: 0.60787 | Test F1: 70.20%



  3%|▎         | 2/64 [00:00<00:17,  3.56it/s]

Train loss: 0.62576 | Train F1: 67.92%
Test loss: 0.60544 | Test F1: 70.20%



  5%|▍         | 3/64 [00:00<00:15,  3.82it/s]

...

Train loss: 0.20136 | Train F1: 92.63%
Test loss: 0.24963 | Test F1: 90.43%



 98%|█████████▊| 63/64 [00:15<00:00,  3.95it/s]

Train loss: 0.20191 | Train F1: 92.26%
Test loss: 0.24221 | Test F1: 91.03%



100%|██████████| 64/64 [00:15<00:00,  4.03it/s]

Train loss: 0.20273 | Train F1: 92.52%
Test loss: 0.24221 | Test F1: 90.73%



In [50]:
validation_pred = model_2(torch.Tensor(validation_features).unsqueeze(dim=1).to(device)).cpu().detach().numpy()

In [51]:
validation_df['pred_2nd'] = np.around(validation_pred[:, 0], 0)

In [52]:
validation_df

,the,to,ect,and,for,of,a,you,hou,in,...,valued,lay,infrastructure,military,allowing,ff,dry,Is_Spam,pred,pred_2nd
5,4,5,1,4,2,3,45,1,0,16,...,0,0,0,0,0,0,0,True,1.0,1.0
15,6,2,1,0,2,0,36,3,1,8,...,0,0,0,0,0,0,0,False,0.0,0.0
17,36,21,6,14,7,17,194,25,5,59,...,0,0,0,0,0,3,0,True,1.0,1.0
20,0,0,1,1,0,0,15,1,0,2,...,0,0,0,0,0,0,0,False,0.0,0.0
22,0,3,6,0,5,0,30,0,2,6,...,0,0,0,0,0,0,0,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,5,8,10,1,0,0,40,3,1,14,...,0,0,0,0,0,0,0,False,0.0,0.0
5136,1,4,1,2,0,1,15,1,0,1,...,0,0,0,0,0,1,0,False,0.0,0.0
5146,0,3,2,0,0,0,7,0,0,1,...,0,0,0,0,0,0,0,False,0.0,0.0
5158,2,1,1,0,1,1,16,0,1,2,...,0,0,0,0,0,1,0,False,0.0,0.0


In [53]:
f1_score(validation_df['pred_2nd'], validation_df['Is_Spam'])

0.8615384615384616

In [54]:
torch.jit.script(model_2).save(os.path.join(os.getcwd(), 'conv_model.pt'))

In [55]:
confusion_matrix(validation_df['pred_2nd'], validation_df['Is_Spam'])

array([[315,  30],
       [  6, 112]])

In [1]:
#Load model and get predictions as numpy array
#Can be run on fresh kernel
import os
import torch
import numpy as np
import pandas as pd

from sklearn import preprocessing

df = pd.read_csv('emails.csv')
device = 'cpu'
scaler = preprocessing.MinMaxScaler()
X_test = scaler.fit_transform(df.loc[:, ~df.columns.isin(['Is_Spam', 'Is_Legit', 'Prediction', 'Email No.'])])
torch.set_default_device('cpu')
final_model = torch.jit.load(os.path.join(os.getcwd(), 'linear_model.pt')).to(device)
final_model.eval()
pred = np.around(final_model(torch.Tensor(X_test).to(device)).cpu().detach().numpy()[:, 0], 0)
pred

array([0., 0., 0., ..., 1., 1., 0.], dtype=float32)